In [1]:
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler 
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import joblib

In [2]:
fifafile= pd.read_csv("C:/Users/gorka//Desktop/Data Analitycs/Ironhack/Unit 2/Fifa Proyect/fifa21_train.csv")
display (fifafile)
fifafile.shape

,ID,Name,Age,Nationality,Club,BP,Position,Team & Contract,Height,Weight,...,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,GK,OVA
0,184383,A. Pasche,26,Switzerland,FC Lausanne-Sport,CM,CM CDM,FC Lausanne-Sport 2015 ~ 2020,"5'9""",161lbs,...,59+1,59+1,59+1,58+1,54+1,54+1,54+1,58+1,15+1,64
1,188044,Alan Carvalho,30,China PR,Beijing Sinobo Guoan FC,ST,ST LW LM,"Beijing Sinobo Guoan FC Dec 31, 2020 On Loan","6'0""",159lbs,...,53+2,53+2,57+2,53+2,48+2,48+2,48+2,53+2,18+2,77
2,184431,S. Giovinco,33,Italy,Al Hilal,CAM,CAM CF,Al Hilal 2019 ~ 2022,"5'4""",134lbs,...,56+2,56+2,59+2,53+2,41+2,41+2,41+2,53+2,12+2,80
3,233796,J. Evans,22,Wales,Swansea City,CDM,CDM CM,Swansea City 2016 ~ 2021,"5'10""",152lbs,...,58+2,58+2,56+2,57+2,58+2,58+2,58+2,57+2,14+2,59
4,234799,Y. Demoncy,23,France,US Orléans Loiret Football,CDM,CDM CM,US Orléans Loiret Football 2018 ~ 2021,"5'11""",150lbs,...,64+2,64+2,64+2,63+2,61+2,61+2,61+2,63+2,15+2,65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11696,232504,B. Böðvarsson,25,Iceland,Jagiellonia Białystok,LB,LB,Jagiellonia Białystok 2018 ~ 2021,"6'1""",168lbs,...,60+2,60+2,63+2,63+2,61+2,61+2,61+2,63+2,16+2,65
11697,214680,G. Gallon,27,France,ESTAC Troyes,GK,GK,ESTAC Troyes 2019 ~ 2022,"6'1""",174lbs,...,26+2,26+2,25+2,24+2,26+2,26+2,26+2,24+2,69+2,70
11698,221489,J. Flores,22,Chile,CD Antofagasta,RM,LM CAM RM,CD Antofagasta 2019 ~ 2024,"5'6""",143lbs,...,44+2,44+2,49+2,45+2,35+2,35+2,35+2,45+2,17+2,67
11699,146717,Anderson Silva,26,Brazil,Barnsley,CM,NaN,Barnsley 2010,"6'2""",179lbs,...,68+0,68+0,66+0,64+0,60+0,60+0,60+0,64+0,25+0,68


(11701, 101)

In [3]:
def clean_data(df):
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_rows', None)
    # I find this OVA calculator: https://sofifa.com/calculator?r=210056&set=true. So columns that I maintain are "Name" and "BP" to use them as a filter. Moreover, I keep the columns that shows the link. 
    df =df.drop(['Club','Joined','Loan Date End','A/W','D/W','ID','Age','Position','Nationality','Team & Contract','Height','Weight','foot','Growth','Value','Wage','Release Clause','Contract','Attacking','Skill','Movement','Power','Mentality','Defending','Goalkeeping','Total Stats','Base Stats', "W/F", 'Hits', 'LS', 'ST', 'RS', 'LW', 'LF', 'CF', 'RF',
       'RW', 'LAM', 'CAM', 'RAM', 'LM', 'LCM', 'CM', 'RCM', 'RM', 'LWB', 'LDM',
       'CDM', 'RDM', 'RWB', 'LB', 'LCB', 'CB', 'RCB', 'RB', 'GK',"SM","BP","Name"], axis=1)
    # Drop some NaN values
    df = df[df['Volleys'].isna()==False]
    # In this case I decided to complete them with the mean. 
    mean_composure = df["Composure"].mean()
    df["Composure"] =df["Composure"].fillna(mean_composure)
    df['IR']=df['IR'].str.replace('★','') #This for replace the star with nothing
    df['IR'] = pd.to_numeric(df['IR'],errors='coerce') # It was string so I convert to numerical
    # Separate categorical and numerical columns
    numerical = df.select_dtypes(include = np.number)
    categorical = df.select_dtypes(include = object)
    return df

In [4]:
fifafile = clean_data(fifafile)

In [5]:
# X and y split
y = fifafile['OVA']
X = fifafile.drop(['OVA'], axis=1)
X_num = X.select_dtypes(include = np.number)
# Normalization. In this case I don´t encode categorical because we maintain them for filtering.
transformer = MinMaxScaler().fit(X_num)
x_normalized = transformer.transform(X_num)
x_normalized = pd.DataFrame(x_normalized, columns=X_num.columns)
# Test train split
X_train, X_test, y_train, y_test = train_test_split(x_normalized, y, test_size=0.2)
lm = linear_model.LinearRegression()
lm.fit(X_train,y_train) # model training
y_pred_linreg=lm.predict(X_test) # model prediction
# # Save the model
# linearmodel = "finalized_model.sav"
# joblib.dump(lm,linearmodel)
# transformersave = "Minmaxscalerfit.sav"
# joblib.dump(transformer,transformersave)

In [6]:
# Validation R2 scores
predictions = lm.predict(X_train)
r2_score_train = r2_score(y_train, predictions)
predictions_test = lm.predict(X_test)
r2_score_test = r2_score(y_test, predictions_test)
print ('train R2: {} -- test R2: {}'.format(r2_score_train,r2_score_test))

train R2: 0.8944400379559475 -- test R2: 0.894130204997839


In [7]:
# Mean Squared Error
train_mse=mean_squared_error(y_train,predictions)
test_mse=mean_squared_error(y_test,predictions_test)

print ('train MSE: {} -- test MSE: {}'.format(train_mse, test_mse))

train MSE: 4.99726547353327 -- test MSE: 4.797523156219962


In [8]:
# Root Mean Squared Error
rmse_train = np.sqrt(mean_squared_error(y_train,predictions))
rmse_test = np.sqrt(mean_squared_error(y_test,predictions_test))
print ('train RMSE: {} -- test RMSE: {}'.format(rmse_train, rmse_test))

train RMSE: 2.2354564351678317 -- test RMSE: 2.190324897411332


In [9]:
# Mean Absolute Error
mae_train = mean_absolute_error(y_train, predictions)
mae_test = mean_absolute_error(y_test, predictions_test)
print ('train MAE: {} -- test MAE: {}'.format(mae_train,mae_test))

train MAE: 1.7502435445454207 -- test MAE: 1.715236293635583


In [10]:
y_test[:5]

9550    70
820     77
7889    72
6534    65
8427    63
Name: OVA, dtype: int64

In [11]:
predictions_test[:5]

array([65.75857418, 74.50591258, 75.87593986, 67.04436989, 57.88969512])

In [12]:
#try this model in another file

In [13]:
fifava= pd.read_csv("C:/Users/gorka//Desktop/Data Analitycs/Ironhack/Unit 2/Fifa Proyect/fifa21_validate.csv")
display (fifava.head())
fifava.shape

,ID,Name,Age,Nationality,Club,BP,Position,Team & Contract,Height,Weight,foot,Growth,Joined,Loan Date End,Value,Wage,Release Clause,Contract,Attacking,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Skill,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Movement,Acceleration,Sprint Speed,Agility,Reactions,Balance,Power,Shot Power,Jumping,Stamina,Strength,Long Shots,Mentality,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Defending,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,W/F,SM,A/W,D/W,IR,PAC,SHO,PAS,DRI,DEF,PHY,Hits,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,GK,OVA
0,219461,E. Palmer-Brown,23,United States,FK Austria Wien,CB,CB,"FK Austria Wien Jun 30, 2021 On Loan","6'2""",194lbs,Right,7,"Feb 8, 2018","Jun 30, 2021",€975K,€5K,€0,"Jun 30, 2021 On Loan",230,47,21,62,60,40.0,228,44,43.0,36,51,54,303,60,68,63.0,63,49.0,288,48,77.0,51,87,25,246,68,62.0,38.0,39.0,39,49.0,200,68,67,65.0,56,11,8,15,13,9,1551,334,2 ★,2★,Low,High,1 ★,64,30,50,50,66,74,34,48+2,48+2,48+2,48+0,47+0,47+0,47+0,48+0,48+2,48+2,48+2,50+2,51+2,51+2,51+2,50+2,59+2,61+2,61+2,61+2,59+2,61+2,67+2,67+2,67+2,61+2,16+2,67
1,221896,D. Avdijaj,22,Kosovo,Heart of Midlothian,CAM,LM CAM,Heart of Midlothian 2020 ~ 2020,"5'8""",154lbs,Right,5,"Jan 20, 2020",NaN,€1.2M,€3K,€2.2M,2020 ~ 2020,298,62,60,44,62,70.0,330,76,68.0,56,60,70,375,77,72,83.0,64,79.0,323,76,62.0,63,47,75,286,72,26.0,64.0,64.0,60,65.0,61,19,23,19.0,53,14,13,9,9,8,1726,358,4 ★,3★,High,Low,1 ★,74,67,62,74,24,57,12,64+2,64+2,64+2,68+0,68+0,68+0,68+0,68+0,68+2,68+2,68+2,67+2,61+2,61+2,61+2,67+2,49+2,47+2,47+2,47+2,49+2,45+2,38+2,38+2,38+2,45+2,17+2,68
2,247428,D. Ochoa,19,United States,Real Salt Lake,GK,GK,Real Salt Lake 2018 ~ 2020,"6'2""",176lbs,Right,17,"Nov 28, 2018",NaN,€120K,€500,€249K,2018 ~ 2020,48,7,5,11,21,4.0,52,6,8.0,8,20,10,165,28,25,33.0,41,38.0,171,40,49.0,22,54,6,76,20,9.0,7.0,26.0,14,31.0,27,8,9,10.0,269,56,52,53,53,55,808,295,2 ★,1★,Medium,Medium,1 ★,56,52,53,55,26,53,3,18+2,18+2,18+2,15+0,17+0,17+0,17+0,15+0,17+2,17+2,17+2,16+2,18+2,18+2,18+2,16+2,16+2,18+2,18+2,18+2,16+2,16+2,18+2,18+2,18+2,16+2,53+2,54
3,255120,N. Kenneh,16,England,Leeds United,CDM,CB CDM RB,Leeds United 2020 ~ 2022,"6'3""",170lbs,Right,23,"Jan 10, 2020",NaN,€160K,€500,€464K,2020 ~ 2022,215,38,31,55,59,32.0,224,51,34.0,38,47,54,275,59,58,56.0,48,54.0,242,48,48.0,60,58,28,230,61,55.0,33.0,40.0,41,59.0,159,53,52,54.0,36,7,5,13,5,6,1381,303,3 ★,2★,Medium,Medium,1 ★,58,34,47,52,53,59,6,46+2,46+2,46+2,47+0,46+0,46+0,46+0,47+0,47+2,47+2,47+2,49+2,49+2,49+2,49+2,49+2,53+2,54+2,54+2,54+2,53+2,53+2,54+2,54+2,54+2,53+2,11+2,55
4,215556,E. Fernandes,24,Switzerland,1. FSV Mainz 05,CDM,CM CDM,1. FSV Mainz 05 2019 ~ 2023,"6'2""",170lbs,Right,5,"Jul 1, 2019",NaN,€2.3M,€13K,€4.3M,2019 ~ 2023,295,57,59,45,78,56.0,327,71,57.0,51,74,74,320,68,66,66.0,64,56.0,337,73,56.0,74,72,62,314,66,78.0,53.0,62.0,55,63.0,211,72,68,71.0,60,12,7,13,15,13,1864,407,4 ★,2★,Medium,Medium,1 ★,67,62,68,70,69,71,45,63+2,63+2,63+2,66+0,66+0,66+0,66+0,66+0,68+2,68+2,68+2,67+2,70+2,70+2,70+2,67+2,70+2,72+2,72+2,72+2,70+2,69+2,68+2,68+2,68+2,69+2,18+2,70


(1999, 101)

In [14]:
fifava = clean_data(fifava)

In [17]:
# X and y split
y2 = fifava['OVA']
X2 = fifava.drop(['OVA'], axis=1)
X_num2 = X2.select_dtypes(include = np.number)
# Normalization. In this case I don´t encode categorical because we maintain them for filtering.
x_normalized2 = transformer.transform(X_num2)
x_normalized2 = pd.DataFrame(x_normalized2, columns=X_num2.columns)

In [19]:
# Validation R2 scores
predictions2 = lm.predict(x_normalized2)
r2_score_train2 = r2_score(y2, predictions2)
predictions_test = lm.predict(X_test)
r2_score_test = r2_score(y_test, predictions_test)
print ('train R2: {} -- test R2: {}'.format(r2_score_train2,r2_score_test))

train R2: 0.8924893914599467 -- test R2: 0.894130204997839


In [20]:
# Mean Squared Error
train_mse2=mean_squared_error(y2,predictions2)
test_mse=mean_squared_error(y_test,predictions_test)

print ('train MSE: {} -- test MSE: {}'.format(train_mse2, test_mse))

train MSE: 4.913351166299565 -- test MSE: 4.797523156219962


In [21]:
# Root Mean Squared Error
rmse_train2 = np.sqrt(mean_squared_error(y2,predictions2))
rmse_test = np.sqrt(mean_squared_error(y_test,predictions_test))
print ('train RMSE: {} -- test RMSE: {}'.format(rmse_train2, rmse_test))

train RMSE: 2.216608031723147 -- test RMSE: 2.190324897411332


In [22]:
# Mean Absolute Error
mae_train2 = mean_absolute_error(y2, predictions2)
mae_test = mean_absolute_error(y_test, predictions_test)
print ('train MAE: {} -- test MAE: {}'.format(mae_train2,mae_test))

train MAE: 1.7480664141393352 -- test MAE: 1.715236293635583
